# **ETL**

## **Librerías**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sqlalchemy import create_engine

In [3]:
from utils import code_clean, code_comm, create_dimension

## **Datos - Cargue**

In [4]:
df_trades = pd.read_csv('./Datos/operaciones.csv', sep=';')
df_countries = pd.read_json('./Datos/paises.json')
df_codes = pd.read_csv('./Datos/productos.csv')

In [5]:
df_parents = df_codes.query('Level == 2').copy()

## **Limpieza y transformación**

#### **Limpiando el código de la partida**

In [6]:
# Eliminamos los valores nulos
df_codes = df_codes[df_codes['Code_comm'].notnull()]

In [7]:
# Ajustamos el código para realizar los cruces
df_codes['Code_Clean'] = df_codes['Code'].apply(lambda x: code_clean(x))
df_codes['Code_comm'] = df_codes['Code_Clean'].apply(lambda x: code_comm(x))

In [8]:
# Cruzamos los codigos con la descripción maestra
df_codes = pd.merge(
    df_codes, 
    df_parents[['Code_comm', 'Description']], 
    how='left', 
    left_on='Code_comm', 
    right_on='Code_comm'
)

In [9]:
# Ajustamos los nombres de los campos
df_codes.rename(
    columns = {'Description_x': 'Description', 'Description_y': 'Parent_Description'}, 
    inplace=True
)

In [10]:
# Cramos un nuevo dataframe con los campos que vamos a utilizar
wf_codes = df_codes[['Code_Clean', 'Description', 'Parent_Description']].copy()

In [11]:
# Creamos un id para cada registro
wf_codes['id_code'] = wf_codes.index + 1

In [12]:
# Ajustamos el formato del código
wf_codes['Code_Clean'] = wf_codes['Code_Clean'].apply(lambda x: int(x))

In [13]:
wf_codes.head()

,Code_Clean,Description,Parent_Description,id_code
0,10011,LIVE ANIMALS; ANIMAL PRODUCTS,LIVE ANIMALS,1
1,10021,LIVE ANIMALS,LIVE ANIMALS,2
2,10100,"Live horses, asses, mules and hinnies",LIVE ANIMALS,3
3,10121,Pure-bred breeding horses,LIVE ANIMALS,4
4,10129,Live horses (excl. pure-bred for breeding),LIVE ANIMALS,5


#### **Limpiando el código de los países**

In [14]:
# Seleccionamos las columnas que vamos a utilizar
wf_countries = df_countries[['alpha-3', 'country', 'region', 'sub-region']].copy()

In [15]:
# Eliminamos los valores nulos
wf_countries = wf_countries[wf_countries['alpha-3'].notnull()]

In [16]:
# Creamos un id para cada registro
wf_countries['id_country'] = wf_countries.index + 1

In [17]:
wf_countries.sample(3)

,alpha-3,country,region,sub-region,id_country
105,MYS,Malaysia,Asia,South-eastern Asia,106
250,NFK,Norfolk Island,Oceania,Australia and New Zealand,251
36,COL,Colombia,Americas,Latin America and the Caribbean,37


#### **DataFrame de Operaciones**

In [18]:
# Combinamos los dataframes
wf = pd.merge(df_trades, wf_codes[['Code_Clean', 'id_code']], how='left', left_on='COMM_CODE', right_on='Code_Clean')
wf = pd.merge(wf, wf_countries[['alpha-3', 'id_country']], how='left', left_on='COUNTRY_CODE', right_on='alpha-3')

In [19]:
# Creamos dataframes intermedios
df_quantity = create_dimension(wf['QUANTITY_NAME'].unique(), 'id_quantity')
df_flow = create_dimension(wf['FLOW'].unique(), 'id_flow')
df_year = create_dimension(wf['YEAR'].unique(), 'id_year')
